In [ ]:
# Import necessary libraries
import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

Loaded decision_tree_model (pickle model)
Loaded linear_regression_model (pickle model)
Loaded random_forest_model (pickle model)


Loaded gru_model (Keras/TensorFlow model)
Loaded svr_model (pickle model)
Loaded slinear_regression_model (pickle model)
Error loading prophet_model: invalid load key, '\x10'.
Loaded lstm_model (Keras/TensorFlow model)
Loaded voting_model (pickle model)
Loaded xgboost_model (pickle model)

Available models:
1. decision_tree_model
2. linear_regression_model
3. random_forest_model
4. gru_model
5. svr_model
6. slinear_regression_model
7. lstm_model
8. voting_model
9. xgboost_model


In [ ]:
models_dir = "Models"  

# Function to load models
def load_models(models_dir):
    """Load all models from the models directory."""
    models = {}
    
    for filename in os.listdir(models_dir):
        file_path = os.path.join(models_dir, filename)
        
        # Skip directories and non-model files
        if os.path.isdir(file_path) or (not filename.endswith('.pkl') and not filename.endswith('.h5')):
            continue
            
        model_name = os.path.splitext(filename)[0]
        
        try:
            if filename.endswith('.pkl'):
                # Load pickle models (scikit-learn models)
                with open(file_path, 'rb') as f:
                    models[model_name] = pickle.load(f)
                print(f"Loaded {model_name} (pickle model)")
                
            elif filename.endswith('.h5'):
                # Load Keras/TensorFlow models
                models[model_name] = keras.models.load_model(file_path)
                print(f"Loaded {model_name} (Keras/TensorFlow model)")
        except Exception as e:
            print(f"Error loading {model_name}: {e}")
    
    return models


models = load_models(models_dir)


print("\nAvailable models:")
for i, model_name in enumerate(models.keys()):
    print(f"{i+1}. {model_name}")

Loaded decision_tree_model (pickle model)
Loaded linear_regression_model (pickle model)
Loaded random_forest_model (pickle model)
Loaded gru_model (Keras/TensorFlow model)
Loaded svr_model (pickle model)
Loaded slinear_regression_model (pickle model)
Error loading prophet_model: invalid load key, '\x10'.


Loaded lstm_model (Keras/TensorFlow model)
Loaded voting_model (pickle model)
Loaded xgboost_model (pickle model)

Available models:
1. decision_tree_model
2. linear_regression_model
3. random_forest_model
4. gru_model
5. svr_model
6. slinear_regression_model
7. lstm_model
8. voting_model
9. xgboost_model
